# Gerenciamento de Parâmetros

Depois de escolher uma arquitetura
e definir nossos hiperparâmetros,
passamos para o ciclo de treinamento,
onde nosso objetivo é encontrar valores de parâmetro
que minimizam nossa função de perda.
Após o treinamento, precisaremos desses parâmetros
para fazer previsões futuras.
Além disso, às vezes desejamos
para extrair os parâmetros
seja para reutilizá-los em algum outro contexto,
para salvar nosso modelo em disco para que
pode ser executado em outro *software*,
ou para exame na esperança de
ganhar compreensão científica.

Na maioria das vezes, seremos capazes de
ignorar os detalhes essenciais
de como os parâmetros são declarados
e manipulado, contando com estruturas de *Deep Learning*
para fazer o trabalho pesado.
No entanto, quando nos afastamos de
arquiteturas empilhadas com camadas padrão,
às vezes precisaremos 
declarar e manipular parâmetros.
Nesta seção, cobrimos o seguinte:

* Parâmetros de acesso para depuração, diagnóstico e visualizações.
* Inicialização de parâmetros.
* Parâmetros de compartilhamento em diferentes componentes do modelo.

Começamos nos concentrando em um MLP com uma camada oculta.


In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2046],
        [0.4232]], grad_fn=<AddmmBackward>)

## Acesso a Parâmetros

Vamos começar explicando como acessar os parâmetros
dos modelos que você já conhece.
Quando um modelo é definido por meio da classe `Sequential`,
podemos primeiro acessar qualquer camada indexando
no modelo como se fosse uma lista.
Os parâmetros de cada camada são convenientemente
localizado em seu atributo.
Podemos inspecionar os parâmetros da segunda camada totalmente conectada da seguinte maneira.


In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2166, -0.3507, -0.1181,  0.2769, -0.1432,  0.2410,  0.0740,  0.2221]])), ('bias', tensor([0.3118]))])


A saída nos diz algumas coisas importantes.
Primeiro, esta camada totalmente conectada
contém dois parâmetros,
correspondendo aos
pesos e vieses, respectivamente.
Ambos são armazenados como *floats* de precisão simples (float32).
Observe que os nomes dos parâmetros
nos permitem identificar de forma única
parâmetros de cada camada,
mesmo em uma rede contendo centenas de camadas.


### Parâmetros Direcionados

Observe que cada parâmetro é representado
como uma instância da classe de parâmetro.
Para fazer algo útil com os parâmetros,
primeiro precisamos acessar os valores numéricos subjacentes.
Existem várias maneiras de fazer isso.
Alguns são mais simples, enquanto outros são mais gerais.
O código a seguir extrai o viés
da segunda camada de rede neural, que retorna uma instância de classe de parâmetro, e
acessa posteriormente o valor desse parâmetro.


In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.3118], requires_grad=True)
tensor([0.3118])


Os parâmetros são objetos complexos,
contendo valores, gradientes,
e informações adicionais.
É por isso que precisamos solicitar o valor explicitamente.

Além do valor, cada parâmetro também nos permite acessar o gradiente. Como ainda não invocamos a *backpropagation* para esta rede, ela está em seu estado inicial.


In [4]:
net[2].weight.grad == None

True

### Todos os Parâmetros de Uma Vez

Quando precisamos realizar operações em todos os parâmetros,
acessá-los um por um pode se tornar tedioso.
A situação pode ficar especialmente complicada
quando trabalhamos com blocos mais complexos (por exemplo, blocos aninhados),
uma vez que precisaríamos voltar recursivamente
através de toda a árvore para extrair
parâmetros de cada sub-bloco. Abaixo, demonstramos como acessar os parâmetros da primeira camada totalmente conectada versus acessar todas as camadas.


In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


Isso nos fornece outra maneira de acessar os parâmetros da rede como segue.


In [6]:
net.state_dict()['2.bias'].data

tensor([0.3118])

### Coletando Parâmetros de Blocos Aninhados

Vamos ver como funcionam as convenções de nomenclatura de parâmetros
se aninharmos vários blocos uns dentro dos outros.
Para isso, primeiro definimos uma função que produz blocos
(uma fábrica de blocos, por assim dizer) e então
combine-os dentro de blocos ainda maiores.


In [7]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # Nested here
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.1150],
        [0.1150]], grad_fn=<AddmmBackward>)

Agora que projetamos a rede,
vamos ver como está organizado.


In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


Uma vez que as camadas são aninhadas hierarquicamente,
também podemos acessá-los como se
indexação por meio de listas aninhadas.
Por exemplo, podemos acessar o primeiro bloco principal,
dentro dele o segundo sub-bloco,
e dentro disso o viés da primeira camada,
com o seguinte.


In [9]:
rgnet[0][1][0].bias.data

tensor([ 0.3746,  0.1523, -0.2427, -0.0837, -0.3223, -0.3808, -0.1753,  0.4000])

## Inicialização de Parâmetros

Agora que sabemos como acessar os parâmetros,
vamos ver como inicializá-los corretamente.
Discutimos a necessidade de inicialização adequada em :numref:`sec_numerical_stability`.
A estrutura de *Deep Learning* fornece inicializações aleatórias padrão para suas camadas.
No entanto, muitas vezes queremos inicializar nossos pesos
de acordo com vários outros protocolos. A estrutura fornece mais comumente
protocolos usados e também permite criar um inicializador personalizado.


Por padrão, o PyTorch inicializa matrizes de ponderação e polarização
uniformemente extraindo de um intervalo que é calculado de acordo com a dimensão de entrada e saída.
O módulo `nn.init` do PyTorch oferece uma variedade
de métodos de inicialização predefinidos.


### Inicialização *Built-in* 

Vamos começar chamando inicializadores integrados.
O código abaixo inicializa todos os parâmetros de peso
como variáveis aleatórias gaussianas
com desvio padrão de 0,01, enquanto os parâmetros de polarização são zerados.


In [10]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0045, -0.0082, -0.0063,  0.0008]), tensor(0.))

Também podemos inicializar todos os parâmetros
a um determinado valor constante (digamos, 1).


In [11]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

Também podemos aplicar inicializadores diferentes para certos blocos.
Por exemplo, abaixo inicializamos a primeira camada
com o inicializador Xavier
e inicializar a segunda camada
para um valor constante de 42.


In [12]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5433, -0.5930, -0.6287, -0.6148])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### Inicialização Customizada

Às vezes, os métodos de inicialização de que precisamos
não são fornecidos pela estrutura de *Deep Learning*.
No exemplo abaixo, definimos um inicializador
para qualquer parâmetro de peso $w$ usando a seguinte distribuição estranha:

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U(-10, -5) & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


Novamente, implementamos uma função `my_init` para aplicar a` net`.


In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 6.2500,  9.0335,  6.4244, -6.6561],
        [-5.9088,  9.8715,  7.8404,  0.0000]], grad_fn=<SliceBackward>)

Observe que sempre temos a opção
de definir parâmetros diretamente.


In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, 10.0335,  7.4244, -5.6561])

## Parâmetros *Tied*


Frequentemente, queremos compartilhar parâmetros em várias camadas.
Vamos ver como fazer isso com elegância.
A seguir, alocamos uma camada densa
e usar seus parâmetros especificamente
para definir os de outra camada.


In [15]:
# Precisamos dar as camadas compartilhadas um nome
# para que possamos referenciar seus parâmetros

#
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# Checar se são os mesmos parâmetros
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Garantindo que são o mesmo objeto ao invés de ter
# apenas o mesmo valor
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


Este exemplo mostra que os parâmetros
da segunda e terceira camadas são amarrados.
Eles não são apenas iguais, eles são
representado pelo mesmo tensor exato.
Assim, se mudarmos um dos parâmetros,
o outro também muda.
Você pode se perguntar,
quando os parâmetros são amarrados
o que acontece com os gradientes?
Uma vez que os parâmetros do modelo contêm gradientes,
os gradientes da segunda camada oculta
e a terceira camada oculta são adicionadas juntas
durante a retropropagação.


## Sumário

* Temos várias maneiras de acessar, inicializar e vincular os parâmetros do modelo.
* Podemos usar inicialização personalizada.


## Exercícios

1. Use o modelo `FancyMLP` definido em :numref:`sec_model_construction` e acesse os parâmetros das várias camadas.
1. Observe o documento do módulo de inicialização para explorar diferentes inicializadores.
1. Construa um MLP contendo uma camada de parâmetros compartilhados e treine-o. Durante o processo de treinamento, observe os parâmetros do modelo e gradientes de cada camada.
1. Por que compartilhar parâmetros é uma boa ideia?


[Discussão](https://discuss.d2l.ai/t/57)


<!--stackedit_data:
eyJoaXN0b3J5IjpbMjI3MjUyNjYwLC03MTk0NTg0ODEsMjUwMT
g4NTk3LC0xNTkzNzg3NzI4LDE2NDI5Nzg1MDFdfQ==
-->
